# Executing Categories

- I like code
- Language significantly influences thought
- Representation is irrelevant
- Representation is crucial
- Different styles

## Choices

### Haskell

- My originating point

```haskell
class Category cat :: o -> o -> Type where
    -- | the identity morphism
    id :: cat a a

    -- | morphism composition
    (.) :: cat b c -> cat a b -> cat a c

```

- Composition is well typed.
- unification / type inferrance does some work for you
- Useful model of polymorphism
- Wacky type level programing or true dependent types.
- This is no more what a category *is* than this encoding of peano numbers are the naturals.
```haskell
data Nat = Zero | Succ Nat
```

- Compiling to categories - http://conal.net/papers/compiling-to-categories/

### Theorem Provers

- https://mathoverflow.net/questions/152497/formalizations-of-category-theory-in-proof-assistants
- https://github.com/agda/agda-categories
- https://github.com/statebox/idris-ct
- https://github.com/jwiegley/category-theory
- https://arxiv.org/pdf/1401.7694.pdf
- https://www.isa-afp.org/


### Computational Category Theory
http://www.cs.man.ac.uk/~david/categories/book/book.ps


### Python
- Ubiquitous
- Popular
- Lots of packages
- https://github.com/oxford-quantum-group/discopy

### Julia

- Acceptable syntax for Scientists and Engineers
- Catlab
- Fascinating package ecosystem.
- A fun assortment of features for the right variety of language wonk
- Easy to implement novel algorithms fast.
- Open minded experimental community.

- https://www.epatters.org/wiki/algebra/computational-category-theory.html
- https://algebraicjulia.github.io/Catlab.jl/latest/

# Computer Functions as a category
 
- Morphisms are functions
- Objects are types
- Base structural combinators + a piles of domain specific atoms

In [ ]:
module Func1
    id = x -> x
    compose(f,g) = x -> g(f(x))
    otimes(f,g) = ((x,y),) -> (f(x),g(y))
    mcopy = x -> (x,x)
    fst = ((x,y),) -> x
    snd = ((x,y),) -> y
    pair(f,g) = x -> (f(x), g(x))
    assoc = (((x,y),z),) -> (x, (y,z))
    braid = ((x,y),) -> (y,x)

    add = ((x,y),) -> x + y # xy -> +(xy...)
    mul = ((x,y),) -> x * y # xy -> *(xy...)
    #spread(f) = (x,) -> f(x...)
    square = compose(mcopy, mul)

    # examples
    display([
        compose(fst,snd)(((:a, :c), :b))
        compose(fst, id)((:a, :b))
        id(:a)
        otimes(id,id)((:a, :b))
    ])
end

### Flat representation
- There are always choices in representation
- On the nose associativity via storing in arrays
- canonical form is nice, no assoc jiggling
- Flatten pointer indirections
- We run into problems where the data we need just isn't there
- isomorphism vs equality


In [ ]:
module NoGo
    id = x -> x
    compose(f,g) = x -> g(f(x))
    fst = () # ??? 
    otimes(f,g) = () #  ???
end

### How to get missing information
- There isn't really any magic. You ned the data you need

1. Encode in types
2. user annotate everything with types. pair(a,b,c,f,g)
3. Wrapper types. Avoids some annotations
4. others

In [10]:
module Func
    id(n) = x -> x
    fst(n,m) = x -> x[1:n] # fst(n) ?
    snd(n,m) = x -> x[end-m:end] #snd(m) ?
    compose(f,g) = x -> g(f(x))
    otimes(f,g) = x -> () # ???
    mcopy = x -> [x ; x]
    pair(f,g) = x -> [f(x) ; g(x)]
    braid(n,m) = x -> [x[end-m:end] ; x[1:n]]

    # examples
    display([
        compose(fst(2,1),snd(1,1))([:a :c :b])
        compose(fst(1,1), id(1))([:a :b])
        id(1)([:a])
        #otimes(id(1),id(1))([:a :b])
    ])
end

4-element Array{Symbol,1}:
 :a
 :c
 :a
 :a

Main.Func

In [4]:
module FuncWrap
    struct FunOb
       ob::Int64 # Fishy ain't it. Alternatively  Array{Type}
    end
    struct FunMorph
       fun
       dom
       cod
    end
    id(n::FunOb) = FunMorph(x -> x, n , n)
    fst(n::FunOb,m::FunOb) = FunMorph(x -> x[1:n.ob], FunOb(n.ob + m.ob), FunOb(n.ob))  # fst(n) ?
    compose(f::FunMorph,g::FunMorph) = FunMorph(x -> g.fun(f.fun(x)),  f.dom, g.cod )
    # and so on
end

Main.FuncWrap

## FinVect

- Linear Operators are the morphism 
- Objects are vector spaces


### Matrices as Morphisms

- Linear Operators represented by matrices
- Vector spaces represented by an integer of dimensions.
- This is *data*


In [11]:
module MatCat
    using LinearAlgebra
    eye(n) = Matrix{Float64}(I,n,n)

    id(n) = eye(n) 
    compose(f,g) = g * f
    function oplus(f,g) #note this monoidal product
        (n,m) = size(f)
        (p,q) = size(g)
        [     f      zeros(n,q) ;
          zeros(p,m)     g      ]
    end
    mcopy(n) = [ eye(n) ;
                 eye(n) ] 
    fst(n,m) = [   eye(n)   zeros(n, m)] 
    snd(n,m) = [zeros(m, n)    eye(m)  ] 

    pair(f,g) = [ f ;
                  g ]

    sum(n) = transpose(mcopy(n))

    display.(
        [
        id(2),
        fst(2,3),
        mcopy(2),
        pair(fst(1,2), snd(1,2))
        ])
end

2×2 Array{Float64,2}:
 1.0  0.0
 0.0  1.0

2×5 Array{Float64,2}:
 1.0  0.0  0.0  0.0  0.0
 0.0  1.0  0.0  0.0  0.0

4×2 Array{Float64,2}:
 1.0  0.0
 0.0  1.0
 1.0  0.0
 0.0  1.0

3×3 Array{Float64,2}:
 1.0  0.0  0.0
 0.0  1.0  0.0
 0.0  0.0  1.0

Main.MatCat

### Functional representations of linear maps

- identical to the computer function implementations
- We can reconstitue the matrix by applying to a basis
- We kill some operations that matrices allow. Gaussian elimination for example
- We can build out of a matrix using the `apply` function
- Equivalently can use the tranpose or multiply from left

- https://github.com/JuliaSmoothOptimizers/LinearOperators.jl
- https://docs.scipy.org/doc/scipy/reference/generated/scipy.sparse.linalg.LinearOperator.html

In [13]:
module LinFunc
    id = x -> x
    fst(n,m) = x -> x[1:n] 
    snd(n,m) = x -> x[end-m:end]
    compose(f,g) = x -> g(f(x))
end
module LinFunc
    import Main.Func

    id = Func.id
    fst = Func.fst
    snd = Func.snd
    compose = Func.compose
    #oplus = Func.otimes
    # and so on
end



Main.LinFunc

## Automatic differentiation as a category
https://www.youtube.com/watch?v=17gfCTnw6uE&feature=youtu.be&ab_channel=Topos Good lord. Conal Elliot is the best.

### Bundle together functions with their Jacobians

- The chain rule is matrix multiplication
- While you're composing functions, compose Jacobians
- So it's a kind of bundling of FunCat & MatCat
- The value in the matrix depend on the basepoint though.
- `(x -> y, dx -> dy)` Nope `x -> (y, dx -> dy)`


In [21]:
module ADMatrix
    import Main.MatCat
    # here are a couple combinators in matrix form
    id(n) = x -> (x, MatCat.id(n))
    compose(f,g) = x -> begin
        (y, df) = f(x)
        (z, dg) = g(y)
        (z, MatCat.compose(df,dg))
    end

    sin = x -> (Base.sin(x), Base.cos(x))
    square = x -> (x * x, 2 * x)
    add = x -> (sum(x), ones(1,length(x)))

    display(compose(square,square)(1))
end




(1, 4)

Main.ADMatrix

### Association and "Hughes Lists"
- Function application is special associated strongly by default
- Lists append cost proprtional to first list.
- Different matrix associativities hace different multiplication cost
- Use LinFunc
- Cayley representation and the Yoneda lemma







In [23]:
module LensRAD
    id = x -> (x, dy -> dy)
    fst(n,m) = x -> (x[1:n] , dy -> [dy ; zeros(m)  ])
    snd(n,m) = x -> (x[end-m:end] , dy -> [zeros(n) ; dx] )
    compose(f, g) = 
        x -> begin
             (y, df) = f(x)
             (z, fg) = g(y)
             (z, df ∘ dg)
             end
    function pair(f,g)
        @assert f.dom == g.dom
        x -> begin
        (y, df) = f(x)
        (z, dg) = g(x)
        (vcat(y,z), dq -> df(dq[1:f.cod]) + dg(dq[end-g.cod:end])   )

                            ( x[1:f.dom]  )
        end
    end
    mcopy(m) = x -> ([x ; x], dxx -> dxx[1:m] + dxx[m+1:end] )

    # the lens domain isn't really necessary. I guess it might changed the stage? Dimension is now known before x values rather than at the same time. That's nice
    mul(m) = x -> (x[1:m] .* x[m+1:end] , dx -> vcat( dx .* x[m+1:end], dx .* x[1:m]  ) )
    add(m) = x -> (x[1:m] .+ x[m+1:end] , dx -> vcat( dx , dx ) ) # sum and dup are dual.

    sin = x -> (sin.(x) , dx -> cos.(x) .* dx)
    cos = x -> (cos.(x) , dx -> -sin.(x) .* dx)
    pow(n) = x -> (x ^ n, dx -> n * dx .* x ^ (n-1) )
end

Main.LensRAD

### How exotic is this?
- It is not. 
- Can Defunctionalize to Wengert tapes
- Can Closure convert to existential lens for object oriented backprop. `{forward :: x -> (e,y), backward :: (e,dy) -> dx}`
- Lenses live on a spectrum of control flow techniques with continuations and coroutines. Question: Can lenses be compiled efficiently?
- Dual numbers are a little wrong. `(x,dx) -> (y,dy)`
- Is this competitive? I don't know and I won't start to believe you unless you work on a serious AD library

### References
- http://conal.net/papers/essence-of-ad/
- https://www.philipzucker.com/reverse-mode-differentiation-is-kind-of-like-a-lens-ii/
- https://t.co/4tjLhB4b4P?amp=1
- https://twitter.com/SandMouth/status/1270409619693875201?s=20
- https://arxiv.org/pdf/1803.10228.pdf
- https://gist.github.com/Keno/4a6507b75288b1fe671e9d1cc683014f - Keno Fischer

## Relations
- And now for something a little different

### Finite Relations
- Objects = finite sets
- Are finite sets just the number of elements they have?
- Morphisms = Relationships between these sets ~ Sets of tuples
- Simplest representation: Array of tuples
- Comprehension notation is da bomb


In [20]:
module FinRel
    id(m) = [(x,x) for x in m ]
    compose(f,g) = [(a,c) for (a,b1) in f for (b2,c) in g if b1 == b2 ]
    fst(a,b) = [ ( [a ; b] , a )  for x in a for y in b ] 
    mcopy(m) = [ (x  , [x ; x]) for x in m ]
    # and so on.


    converse(f) = [(y,x) for (x,y) in f ]
    #trans(f) = [ (a,(b,c))  ]
    
    x ⊆ y = all( [(a,b) ∈ y for (a,b) in x] )

    meet(x,y) = [ (a,b) for (a,b) in x if (a,b) ∈ y ]
    join(y,x) = [ (a,b) for (a,b) in x if (a,b) ∈ y ]
    ∨(x,y) = join(x,y)
    ∧(x,y) = meet(x,y) 
    bottom = []
end

Main.FinRel

## Other representations

- Finite relations are obviously executable.
- Question is efficiency. What queries?

- Sets of tuples
- Boolean matrices
- BDDS
- `a -> [b]` Powerset functions
- DataFrames
- Databases

- Set representation -> Relation representation

- Lattices - Executable subcategories of Rel
    - Intervals
    - Octagons
    - Polyhedra
    - Convex Sets
    - Term Patterns
    - Linear Subspaces


### Reference



- https://www.cambridge.org/core/books/relational-mathematics/8CB9CE4F196319222E8991D909AA2F87 - Relational Mathematics - Gunther Schmidt
- Pixel Arrays - http://math.mit.edu/~dspivak/informatics/PixelArrays--SpivakDobsonKumari.pdf 
- https://www.philipzucker.com/a-short-skinny-on-relations-towards-the-algebra-of-programming/
- http://www4.di.uminho.pt/~jno/ps/pdbc.pdf
- https://themattchan.com/docs/algprog.pdf
- http://www.philipzucker.com/computational-category-theory-in-python-i-dictionaries-for-finset/ - A similar approach for FinSet
- https://github.com/AlgebraicJulia/Catlab.jl/blob/master/src/categorical_algebra/FinSets.jl
- http://www.cas.mcmaster.ca/~kahl/
- https://github.com/AlgebraicJulia/Catlab.jl/blob/master/src/categorical_algebra/FinRelations.jl


## Linear Relations

- The importance of Linear Maps cannot be overtstated 
- Linear Relations are almost as important

Examples:
- Control Systems - 
- Circuits
- Discretized linear PDEs and ODEs

Implementation
- Generators - Good for projection, union
- Relations - Good for conjunction
- Range and Nullspace
- SVD for numerical


In [ ]:
id(n) = [eye(n) -eye(n)]

struct LinRel
    left
    right
end


id(n) = LinRel(eye(n), -eye(n))


function compose(x::LinRel,y::LinRel) 
    # extract sizes of different matrices
    (m, n) = size(x.left)
    (n1, x) = size(x.output)
    @assert n1 == n
    ny, yi = size(y.input)
    ny1, yo = size(y.output)
    @assert ny1 == ny
    
    # combine constraints
    A = [ x.input      x.output zeros(nx,yo) ;
          zeros(ny,xi) y.input  y.output     ]
    # convert to range representation
    B = nullspace(A)
    # project in range representation
    projB = [B[1:xi       ,:] ;
             B[xi+yi+1:end,:] ]
    # return to nullspace representation
    C = Base.transpose(nullspace(Base.transpose(projB)))
    return LinRel( C[:, 1:xi] ,C[:,xi+1:end] )
end

# basically the direct sum. The monoidal product of linear relations
function otimes( x::LinRel{T}, y::LinRel{T}) where {T} 
    nx, xi = size(x.input)
    nx1, xo = size(x.output)
    @assert nx1 == nx
    ny, yi = size(y.input)
    ny1, yo = size(y.output)
    @assert ny1 == ny
    return LinRel(    [ x.input                zeros(nx,yi);
                       zeros(ny,xi)           y.input               ],
                      [x.output               zeros(nx,yo);
                       zeros(ny,xo)     y.output               ])
    
end

- Circuit Diagrams are string diagrams
- Control structures and feedback loops are string diagrams
![image.png](lqr.png)


# Layin' it all out there: A second style for linear relations

- Eager projection is a lot to ask. Projection ~ Quantifier elimination
- Lazily build up problem
- Query solver at the end
- In our case a good solver will


In [24]:

module LinRel2Sketch
    struct LinRel
        left
        hidden
        right
    end

    id(n) = LinRel(eye(n), zeros(n,0),  -eye(n))
    function compose(f,g)
        LinRel( [ f.left ; 0 ; 0]   ,  
                [f.right f.middle   0        0 ;
                   0        0       g.left   g.middle ;
                   eye(n)   0       -eye(n)  0 ]   
               , [0 ; g.right ; 0] )    
    end

end



Main.LinRel2Sketch

### Reference 

- https://www.philipzucker.com/linear-relation-algebra-of-circuits-with-hmatrix/
- http://www.philipzucker.com/categorical-lqr-control-with-linear-relations/
- http://www.philipzucker.com/solving-the-laplace-equations-with-linear-relations/
- https://www.philipzucker.com/checkpoint-implementing-linear-relations-for-linear-time-invariant-systems/

## Point Freeing Pointful DSLs

- A cookbook recipe for DSLs that expose Variables

- JuMP
- cvxpy
- z3
- Sympy
- Homotopy Continuation


1. Thunk or Carry environment
2. create input and output variables
3. Create new side stuff (like constraints)
4. Output tuple of input/output variables + side stuff generated

- If you like monads, you can think of this as either a writer or state monad depending on the case
- Similar to the above in the you can only solve in one shot.
- Lots of fresh vars. Smart DSL?

## Optimization Problems

idea : Relations (feasible set) + additive objective functions = Category

- http://www.philipzucker.com/a-sketch-of-categorical-relation-algebra-combinators-in-z3py/
- http://www.philipzucker.com/categorical-combinators-for-convex-optimization-and-model-predictive-control-using-cvxpy/
- http://www.philipzucker.com/categorical-combinators-for-graphviz-in-python/

In [ ]:
module JumpCat
    using JuMP
    id(m) = model -> begin
        x = @variable(model,[1:m])
        (x, x, 0)
        end
  compose(f,g) =
     model -> begin 
            (x,y, o1) = M.f(model)
            (y1, z, o2) = N.f(model)
            @constraint(model,  y1 .== y)
            (x,z, o1 + o2)
            end

  otimes(f, g) = 
     model -> begin
            (x,y, o1) = f.f(model)
            (a,b, o2) = g.f(model)
            ( [x ; a], [y ; b ], o1 + o2 )
        end

  function run(f, model) = 
     (input,ouput, obj) = f(model)
     @objective(model, ob)
     solve(model)
end




### Speculative Work

- Module Relations
- Semialgebraic Relations
- Theorem Proving For Catlab
- Polyhedral Relations
- iterative LQR as a lens
- Graphs

